In [88]:
# Import Azure OpenAI
import os
from openai import AzureOpenAI
BASE_URL = "https://__.openai.azure.com"
API_KEY = "412~"
DEPLOYMENT_NAME = "gpt-4"

llm = AzureChatOpenAI(
    azure_endpoint=BASE_URL,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
)

In [89]:
from langchain_core.messages import HumanMessage
message = HumanMessage(
    content="Translate this sentence from English to Korean. I love programming."
)
llm([message])

AIMessage(content='나는 프로그래밍을 사랑해.', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 19, 'total_tokens': 32}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3dd7d1da-1536-440b-9ab4-1e356fd53520-0')

In [104]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1, openai_api_key="sk-")

In [105]:
from langchain_community.document_loaders import PyPDFLoader
pdf_filepath = 'e:/data/차세대 한국형 스마트팜 개발.pdf'
loader = PyPDFLoader(pdf_filepath)
documents  = loader.load()

In [106]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)


In [107]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(documents=docs, embedding=embeddings)

In [115]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

# Adapt if needed
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template("""다음 대화와 후속 질문이 주어지면 후속 질문을 독립형 질문으로 바꾸어 표현해줘.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:""")

qa = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=db.as_retriever(),
                                           condense_question_prompt=CONDENSE_QUESTION_PROMPT,
                                           return_source_documents=True,
                                           verbose=False)

In [116]:
chat_history = []
query = "한국형 스마트팜이 뭐야?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

한국형 스마트팜은 사물인터넷, 빅데이터, 인공지능, 로봇 등을 활용하여 농산물의 생육환경을 최적상태로 관리하고 노동력 절감과 생산성 향상을 구현하는 효율적인 농업형태를 말합니다. 이는 농업의 고령화와 높은 노동강도로 인해 청년인구의 유입이 어려운 현상을 해결하는 대안으로 제시되었습니다. 

차세대 한국형 스마트팜 기술개발 프로젝트는 4기관 19개 전담부서가 협업하여 핵심 요소 및 원천 기반기술의 확보를 위해 연구 역량을 집중하고 있습니다. 이 프로젝트는 기술수준별로 스마트팜 모델을 3가지 단계로 구분하여 개발을 추진하고 있으며, 이를 통해 노동력과 농자재의 사용을 줄이고, 생산성과 품질을 제고하며 농가소득과 연계하고, 영농현장의 애로와 연관 산업의 문제를 동시에 해결하는 것이 목표입니다.


In [117]:
chat_history = []
query = "필요한 ICT 기술은?"
result = qa({"question": query, "chat_history": chat_history})
print(result["answer"])

농업분야에서는 스마트 기기를 농사에 효과적으로 접목할 수 있는 ICT 기술이 필요하다. 이에는 사물인터넷, 빅데이터, 인공지능, 로봇 등이 포함되며, 이들 기술을 활용한 스마트팜 연구개발이 중요하다. 또한, ICT 기기들의 규격화와 표준화 역시 필요하다. 특히, 시설원예 및 축산에 사용되는 각종 센서 및 제어기에 대하여 형식과 통신방식 등을 통일하는 표준화 작업이 추진되고 있다.
